In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load your dataset
df = pd.read_csv('Darknet.csv')

In [2]:
# Encode the labels to binary format
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'].replace(['Non-Tor', 'NonVPN'], 'Normal').replace(['Tor', 'VPN'], 'Malicious'))

In [3]:
import numpy as np
# Select relevant features
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
X = df[numeric_features].drop(['Src Port', 'Dst Port', 'Protocol'], axis=1)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)

In [4]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['Label'], test_size=0.3, random_state=42, stratify=df['Label'])

In [5]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [6]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])



Epoch 1/100


2023-12-15 07:37:45.652049: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2477/2477 [==============================] - 2s 839us/step - loss: 0.0451 - accuracy: 0.9874 - val_loss: 4.1870e-04 - val_accuracy: 0.9999
Epoch 2/100
2477/2477 [==============================] - 2s 798us/step - loss: 0.0041 - accuracy: 0.9994 - val_loss: 2.8478e-04 - val_accuracy: 0.9999
Epoch 3/100
2477/2477 [==============================] - 2s 821us/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 6.7905e-04 - val_accuracy: 0.9999
Epoch 4/100
2477/2477 [==============================] - 2s 775us/step - loss: 8.1396e-04 - accuracy: 0.9999 - val_loss: 0.0011 - val_accuracy: 0.9999
Epoch 5/100
2477/2477 [==============================] - 2s 770us/step - loss: 0.0030 - accuracy: 0.9997 - val_loss: 0.0012 - val_accuracy: 0.9999
Epoch 6/100
2477/2477 [==============================] - 2s 824us/step - loss: 0.0037 - accuracy: 0.9998 - val_loss: 8.8708e-04 - val_accuracy: 0.9999
Epoch 7/100
2477/2477 [==============================] - 2s 758us/step - loss: 0.0014 - accuracy: 0.9999 - val

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

   1/1327 [..............................] - ETA: 20s - loss: 5.5597e-21 - accuracy: 1.0000

1327/1327 [==============================] - 1s 413us/step - loss: 4.3119e-04 - accuracy: 1.0000
Test accuracy: 0.9999764561653137


In [8]:
#test with holdout data
df = pd.read_csv('holdout_set_mixed.csv')
df['Label'] = le.transform(df['Label'].replace(['Non-Tor', 'NonVPN'], 'Normal').replace(['Tor', 'VPN'], 'Malicious'))
X = df[numeric_features].drop(['Src Port', 'Dst Port', 'Protocol'], axis=1)
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(X.mean(), inplace=True)
X_scaled = scaler.transform(X)
y = df['Label']
loss, accuracy = model.evaluate(X_scaled, y)
print(f'Holdout accuracy: {accuracy}')

1/1 [==============================] - 0s 16ms/step - loss: 2.3209e-21 - accuracy: 1.0000
Holdout accuracy: 1.0


In [9]:
# Optionally save the model
model.save('darknet_model.h5')